#**Neural Networks**

Over the past number of steps you have learned how to use PCA, Lasso and Ridge regression to reduce the dimensionality of your datasets. One of the underlying assumptions with these methods is that the compression or dimesionality reduction is linear in nature.  PCA effectively creates new latent variables that are a new representations of the orginal data but of a significantly lower dimension. These techniques can be very useful when dealing with variables that are highly correlated or when you just want to simply reduce the volume of your input variables in order to improve the processing spead of your algorithms.

So by now you may have come across the concept of Neural Networks. If you haven't then have a look at this [site](https://pathmind.com/wiki/neural-network#define). Neural Networks are algorithms that are loosely modeled on the human brain. The concept can be best described in Figure 1 below.

<img src="https://www.computing.dcu.ie/~amccarren/mcm_images/neural_network.png" alt="Drawing" style="width:10px height:10px;"/>


Figure 1 (Source: [Wikipedia](https://en.wikipedia.org/wiki/ ))

You will definetly come across neural networks in other courses. They can can be used to build prediction models and as part of Autoencoders models which are used to reduce datasets and remove noise from our datasets.
The principle behind them is that the input or our $X$ data is passed into the system through the input nodes. An input function is applied and this information is then transfered to the hiddend nodes, where an activation function transfers that information to the output function, Figure 2.

<img src="https://www.computing.dcu.ie/~amccarren/mcm_images/perceptron_node.png" alt="Drawing" style="width:10px height:10px;"/>

[Figure 2](https://pathmind.com/wiki/neural-network#define)

The objective is to minimize the difference between the outputs and the results of the activation function, by optimising the input and hidden node weights. Normally, optimization is done using algorithms such as [gradiant descent](https://en.wikipedia.org/wiki/Gradient_descent). Neural Networks will introduce a level of non-linearity and complexity into our data models, thus allowing for greater predictive power. The architecture of these systems can vary with the number of nodes in the hidden layer and the extension to deep learning architectures can be achieved through the addition of multiple hidden layers. In reality, this process will be equivalent to a linear regression model if we remove the hidden nodes and appply a linear activation function.


Now we are going to implement a simple neural network regressor on the California housing dataset. It is important that you understand this simple example as you will need these learnings to build and Autoencoder which is the next step.


In [2]:
import tensorflow
# print(tensorflow.__version__)

In [ ]:
!pip install scikit-learn==1.3.2

You will notice below that I have put "tensorflow.python." in front of the Keras library requests. Don't forget to do this. You will also probably have read alot about Theano, Keras and other machine learning libraries. The benifit of using Keras is that is is quite intuative to use. I won't go into the differences between all the libraries but if your interested go [here](https://www.guru99.com/deep-learning-libraries.html)

Now the code below takes the scikit learn California dataset in and then standardizes it. Unlike ordinary regression that we used in earlier topics, Neural Networks requires that the input variables be standardised as the optimization algorithms will be biased towards variables with larger variance. We are also going to import a few other Scikit learn packages which can help improve predictions.

In [4]:
from sklearn.datasets import fetch_california_housing
import keras
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier, KerasRegressor
import numpy as np


In [5]:
data=fetch_california_housing()
scaler = StandardScaler()

#data = [[0, 0], [0, 0], [1, 1], [1, 1]]
X=scaler.fit_transform(data.data)

#X=data.data
y=data.target


In the next piece of code we are going to break our dataset up into a training and test datasets. Now if your not familar with these terms go to [Toward Science](https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7) for a deeper explaination. You will see the term "Validation" set as well, which in the example below refers to the "kfold" term. Training sets our blocks of data that we "train" our algorithm on. They are effectively the block of data that we use to calculate the weights or the paramters. Normally, the training set is between 70 to 80\% of the orignal dataset. The test set is a proportion of data that we keep back to test our algorithm. We should never use this data to calculate our weights and you should only use it once on any particular model that you develop.

In [ ]:
from sklearn.model_selection import train_test_split
print(X.shape)
print(data.feature_names)
print(data.DESCR)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(y_test)

The neural network needs to start with some weights and then iteratively update them to better values. The term kernel_initializer is a fancy term for which statistical distribution or function to use for initialising the weights. The term "activation" is where we specify the function that is going to transfer the input into the layer in question to its directed output.In the example below we put our input data into nodes which are then transferred to the hidden layer of 5 nodes using a rectified linear unit (relu) function. It has the form :

>> $f(x)=x^+=max(0,x)$ where $x$ is the input neuron

This function has been found to have benifical properties over other functions such as the logistic sigmoid or tanh functions.

Now we are going to build our Neural Network. In it we will have 8 input nodes which will be connected to 5 hidden nodes which will then be connected to one output node, the price of houses or $y$. The diagram at the end of this code should show you how they are connected.

In [ ]:
# Regression Example With California Dataset: Baseline

# define base model
def baseline_model():
	# create model
	model = keras.models.Sequential()
	model.add(keras.layers.Dense(5, input_dim=8, kernel_initializer='normal',use_bias=True,
	                activation='relu'))
	model.add(keras.layers.Dense(1, kernel_initializer='normal',use_bias=True,activation='linear'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model
# evaluate model

model=baseline_model()
print(model.summary())


In the previous piece of code we built the framework of our Neural Network. With the KerasRegressor method we are going to add a few more specifications to our Neural Network. The first thing we do is tell KerasRegressor that we are going to use the framework specfied in the "baseline_model" function. The following terms are important to understand as they come up all the time:

* Sample is a single row of data. In our case it corresponds to the data for one house.

* Batch size is a hyperparameter that defines the number of samples to work through before updating the internal model parameters.

* Epoch is a complete pass through the dataset. It is set up as a hyperparamter and comprises of one or more batches.

The verbose term is just a display parameter for Keras. O displays nothing, verbose = 1 will show a progress bar and verbose=2 will show the Epoch the gradient descent algorithm is currently in.

In [ ]:
estimator = KerasRegressor(model=baseline_model, epochs=10, batch_size=64, verbose=1)

print(estimator.__class__)

estimator.fit(X_train, y_train)

kfold = KFold(n_splits=5)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)

print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

prediction = estimator.predict(X_test)


In [ ]:
print(results)

Note: Your results will differ each time you run this as you are effectively solving a nonlinear equation each time.

Note: The mean squared error is negative because scikit-learn inverts so that the metric is maximized instead of minimized. You can ignore the sign of the result.

The result reports the mean squared error including the average variance across all 5 folds of the cross validation evaluation.

We also produce our predictions on the test set and plot the actual versus the predicted. We should in theory see a $45^o$ line.

In [ ]:
plt.style.use('ggplot')  # or 'classic', 'bmh', etc.

plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.scatter(y_test, prediction, marker='o')
plt.show()

Now just like linear models we can use regularisation to reduce the number of parameters in our model. You can use L1 and L2 regularisation in Keras and it also allows you to specify your own [regularisation function](https://keras.io/regularizers).

>>Consider running the example a few times and compare the average performance.

>> Can you find another metric that will demonstrate how we are performing?

>> In this example we were fortunate to have all continous variables. What would you do if you had categorical variables?

>> Did you notice the number of parameters in the model description. Can you explain why we have 76 weights/parameters?

Put your thoughts on the comments board.



